In [ ]:
import pandas as pd
import numpy as np

# ==========================================
# 1. EXTRACT (Extracción)
# ==========================================
file_path = 'consumo_energia_electrica.csv'

print(">>> Cargando archivo CSV...")
# Leemos intentando utf-8 primero, luego latin1 si falla
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    print("Aviso: UTF-8 falló, intentando con Latin-1...")
    df = pd.read_csv(file_path, encoding='latin1')

>>> Cargando archivo CSV...


In [ ]:
# ==========================================
# 2. TRANSFORM (Transformación y Limpieza)
# ==========================================
print(">>> Iniciando limpieza de datos...")

# --- Limpieza de Strings ---
# 1. ATARIFA: Tenía muchos espacios duros (\u00a0) y espacios normales al final.
# Reemplazamos \xa0 por espacio normal y luego hacemos strip()
if 'ATARIFA' in df.columns:
    df['ATARIFA'] = df['ATARIFA'].astype(str).str.replace(u'\xa0', ' ', regex=False).str.strip()

# 2. RAZON_SOCIAL y otros textos: Quitamos espacios extra al inicio/final
cols_texto = ['RAZON_SOCIAL', 'COD_EMPRESA', 'GRUPO', 'USO', 'COD_SIS_ELECTR']
for col in cols_texto:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# --- Creación de Modelos (Tablas) ---

# TABLA 1: DIMENSIÓN EMPRESAS
df_empresas = df[['COD_EMPRESA', 'RAZON_SOCIAL', 'GRUPO']].drop_duplicates()

# TABLA 2: DIMENSIÓN TARIFAS
df_tarifas = df[['COD_TARIFA', 'ATARIFA', 'USO']].drop_duplicates()

# TABLA 3: HECHOS CONSUMO
# Seleccionamos columnas y aseguramos tipos numéricos
df_consumo = df[['MES_FACTURACION', 'FECHA_EMISION', 'FECHA_CORTE',
                 'COD_EMPRESA', 'COD_TARIFA', 'COD_SIS_ELECTR',
                 'SUMINISTROS', 'PROMEDIO_CONSUMO']].copy()

# Manejo de nulos en métricas (rellenar con 0 si existen)
df_consumo['SUMINISTROS'] = df_consumo['SUMINISTROS'].fillna(0)
df_consumo['PROMEDIO_CONSUMO'] = df_consumo['PROMEDIO_CONSUMO'].fillna(0)

>>> Iniciando limpieza de datos...


In [ ]:
# ==========================================
# 3. LOAD (Carga / Generación de Archivos)
# ==========================================
print(">>> Generando archivos JSON...")

# force_ascii=False es CLAVE para que salgan tildes (ó) y no códigos (\u00f3)
df_empresas.to_json('dim_empresas.json', orient='records', lines=False, force_ascii=False, indent=4)
df_tarifas.to_json('dim_tarifas.json', orient='records', lines=False, force_ascii=False, indent=4)
df_consumo.to_json('fact_consumo.json', orient='records', lines=False, force_ascii=False, indent=4)

print("¡ETL Completado con éxito!")
print(f" - dim_empresas.json: {len(df_empresas)} registros")
print(f" - dim_tarifas.json: {len(df_tarifas)} registros")
print(f" - fact_consumo.json: {len(df_consumo)} registros")

# Verificación visual rápida de la limpieza
print("\nEjemplo de Tarifa limpia:")
print(df_tarifas['ATARIFA'].head(1).values)

>>> Generando archivos JSON...
¡ETL Completado con éxito!
 - dim_empresas.json: 24 registros
 - dim_tarifas.json: 61 registros
 - fact_consumo.json: 1150479 registros

Ejemplo de Tarifa limpia:
['1E-BT No Residencial']
